# LLama-Index Demo

In [59]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


from llama_index.embeddings import HuggingFaceEmbedding

---

# Local Embedding Model

In [13]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Documents
Document contains an
- id
- embedding (None)
- Default metadata: file_path, file_name, file_type, timestamps
- text

In [14]:
documents = SimpleDirectoryReader("data").load_data()

# ServiceContext

In [54]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


LLM is explicitly disabled. Using MockLLM.


# Vector Store Index
- Index build on top of vector store
- Defaults
	- Stores everything in memory, but can persist to disk or be connected with VectorStore
	- Inserts vectors in batches of 2048 nodes

In [55]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

----

# Retriever
Fetches most relevant context given a user query, containing ...
- text
- score

In [64]:
retriever = index.as_retriever(similarity_top_k=1)
retrieved_nodes = retriever.retrieve("When is Arnold Schwarzenegger born?")

print("Retrieved nodes:")
for node in retrieved_nodes:
    print(f"Score {node.score}: {node.text}")


Retrieved nodes:
Score 0.8382571492180423: This is a test file. Arnold Schwarzenegger is born in 1947.


---

# Query Engine
Returns a Response object, containing ...
- response
- source_nodes
- metadata

In [20]:
query_engine = index.as_query_engine()

response = query_engine.query("When is Arnold Schwarzenegger born?")
print(response)

Context information is below.
---------------------
file_path: data/file_1.txt

This is a test file. Arnold Schwarzenegger is born in 1947.

file_path: data/file_2.txt

This is another txt file
---------------------
Given the context information and not prior knowledge, answer the query.
Query: When is Arnold Schwarzenegger born?
Answer: 


# Store index to disk

In [58]:
index.storage_context.persist("data/vector_index")